In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm

In [17]:
class ThompsonSampleGauss:
    """
    A class to perform Gaussian Thompson Sampling on the multi-armed bandit problem.
    """
    
    def __init__(self, bandits: int, means: np.ndarray, stds: np.ndarray, pulls: int) -> None:
        """
        Initializes the ThompsonSampleTest class.

        Args:
            bandits: the number of arms in the bandit problem.
            means: a numpy array representing the true means of the bandits.
            stds: a numpy array representing the true standard deviations of the bandits.
            pulls: the number of times to pull the arms of the bandit problem.
        """
        self.bandits = bandits
        self.prior_means = np.zeros(bandits)
        self.prior_stds = np.ones(bandits) * 1000
        self.actual_means = means
        self.actual_stds = stds
        self.pulls = pulls
        self.samples = np.zeros(bandits)
        self.rewards = np.zeros(bandits)
        self.mean_sequences = {n : [] for n in range(bandits)}
        self.stds_sequences = {n : [] for n in range(bandits)}
        
        for n in range(bandits):
            self.mean_sequences[n].append(0)
        for n in range(bandits):
            self.stds_sequences[n].append(1)
        
    def get_choice(self) -> int:
        """
        Samples the prior distributions to select the arm to pull. 
        Note that we sample it twice to get both a mean and a standard deviation.        
        
        Returns:
            The index of the arm to pull.
        """       
        sample_means = norm.rvs(self.prior_means, self.prior_stds)
        sample_stds = np.abs(norm.rvs(self.prior_means, self.prior_stds))
        
        return np.argmax(norm.rvs(sample_means, sample_stds))
    
    def get_reward(self, choice: int) -> float:        
        """
        Samples the distribution of the arm chosen to get a reward.

        Args:
            choice: The index of the arm chosen.
            
        Returns:
            The reward obtained from the chosen arm.
        """        
        reward = norm.rvs(self.actual_means[choice], self.actual_stds[choice])
        
        return reward
    
    def update_prior(self, choice: int, reward: float) -> None:
        """
        Updates the prior distribution for the chosen arm.

        Args:
            choice: The index of the arm chosen.
            reward: The reward obtained from the chosen arm.
        """       
        self.samples[choice] += 1
        self.prior_means[choice] = (self.prior_means[choice] * (self.samples[choice] - 1) + reward) / self.samples[choice]
        self.prior_stds[choice] = np.sqrt((self.prior_stds[choice]**2 * (self.samples[choice] - 1) + (reward - self.prior_means[choice])**2) / self.samples[choice])
        
        self.mean_sequences[choice].append(self.prior_means[choice])
        self.stds_sequences[choice].append(self.prior_stds[choice])
                                          
    def do_it(self) -> None:
        """
        Performs the Thompson Sampling algorithm for the specified number of pulls.
        """        
        for i in range(self.pulls):

            choice = self.get_choice()
            reward = self.get_reward(choice)
            
            self.rewards[choice] += reward            
            self.update_prior(choice, reward)
    
    def compute_regret(self) -> None:
        """
        Calculates the fraction of the expected total rewards if the best
        distribution was chosen every time.
        
        Returns: the fraction of maximum expected returns
        """
        
        return sum(self.rewards) / (self.pulls * max(self.actual_means))

In [20]:
# generating random sets of means and standard deviations
means = norm.rvs(100, 125, size=100)
stds = np.abs(norm.rvs(52, 25, size=100))

# instantiating the class
test = ThompsonSampleGauss(len(means), means, stds, 500)

# running the thompson sampling algorithm
test.do_it()

# converged?
print("Found best:", np.argmax(means) == np.argmax(test.prior_means))

# total regret
print("Fraction of MaxEV achieved:", round(test.compute_regret(), 4))

Found best: True
Fraction of MaxEV achieved: 0.6057


In [21]:
# lets see a dataframe 
df = pd.DataFrame({"final_est_mean": test.prior_means,
                   "actual_mean" : means,
                   "final_est_std" : test.prior_stds,
                   "actual_std" : stds,
                   "sample_count" : test.samples})

df.sort_values(by='final_est_mean', ascending=False).head(15)

,final_est_mean,actual_mean,final_est_std,actual_std,sample_count
82,493.532210,490.962854,31.063451,31.808530,115.0
62,359.075527,355.836892,68.426154,80.981084,46.0
27,330.223606,345.150203,46.329149,43.911516,29.0
83,328.090056,325.279288,53.886422,47.027748,43.0
84,323.365995,331.007807,42.271014,48.503217,27.0
18,298.031242,303.973060,11.917110,15.108281,23.0
61,291.822799,310.956269,29.688271,46.920082,15.0
79,280.737818,279.895355,34.902089,32.798953,24.0
47,278.002979,287.768427,42.550748,71.790193,17.0
11,267.702491,257.992341,64.998054,67.814213,15.0
